In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT"))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

In [3]:
bigbrotr = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    dbname=DB_NAME
)

In [4]:
def get_schema_overview(conn):
    q = '''
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    '''
    return pd.read_sql(q, conn)

schema_df = get_schema_overview(bigbrotr)
for table_name in schema_df['table_name'].unique():
    df = schema_df[schema_df['table_name'] == table_name].drop(columns='table_name')
    display(table_name.capitalize())
    print(df.to_markdown(index=False))
    print("\n")

'Events'

| column_name   | data_type   |
|:--------------|:------------|
| id            | character   |
| pubkey        | character   |
| created_at    | bigint      |
| kind          | integer     |
| tags          | jsonb       |
| content       | text        |
| sig           | character   |




'Events_relays'

| column_name   | data_type   |
|:--------------|:------------|
| event_id      | character   |
| relay_url     | text        |
| seen_at       | bigint      |




'Relay_metadata'

| column_name        | data_type   |
|:-------------------|:------------|
| relay_url          | text        |
| generated_at       | bigint      |
| connection_success | boolean     |
| nip11_success      | boolean     |
| openable           | boolean     |
| readable           | boolean     |
| writable           | boolean     |
| rtt_open           | integer     |
| rtt_read           | integer     |
| rtt_write          | integer     |
| name               | text        |
| description        | text        |
| banner             | text        |
| icon               | text        |
| pubkey             | text        |
| contact            | text        |
| supported_nips     | jsonb       |
| software           | text        |
| version            | text        |
| privacy_policy     | text        |
| terms_of_service   | text        |
| limitation         | jsonb       |
| extra_fields       | jsonb       |




'Relays'

| column_name   | data_type   |
|:--------------|:------------|
| url           | text        |
| network       | text        |
| inserted_at   | bigint      |




In [5]:
def bytes_to_gb(b):
    return b / (1024 ** 3)

cursor = bigbrotr.cursor()

# Recupera tutte le tabelle nel tuo schema pubblico
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
""")
tables = [row[0] for row in cursor.fetchall()]

print("📊 Analisi dello spazio per tabella:\n")

for table in tables:
    print(f"🧾 TABELLA: {table}")
    
    # Numero righe
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    total_rows = cursor.fetchone()[0]
    print(f"Numero righe: {total_rows}")

    if total_rows == 0:
        print(" (Tabella vuota)\n")
        continue

    # Calcola dimensione media per colonna
    cursor.execute(f"""
        SELECT string_agg(
            'AVG(pg_column_size(\"' || column_name || '\")) AS \"' || column_name || '\"',
            ', '
        )
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = 'public'
    """, (table,))
    agg_expr = cursor.fetchone()[0]

    avg_size_query = f"SELECT {agg_expr} FROM {table}"
    cursor.execute(avg_size_query)
    avg_sizes = cursor.fetchone()
    colnames = [desc.name for desc in cursor.description]

    print("{:<20} {:>12} {:>12}".format("Colonna", "Avg (bytes)", "Totale (GB)"))
    total_data_bytes = 0

    for i, col in enumerate(colnames):
        avg_bytes = float(avg_sizes[i])
        total_bytes = avg_bytes * total_rows
        total_data_bytes += total_bytes
        print("{:<20} {:>12.2f} {:>12.2f}".format(col, avg_bytes, bytes_to_gb(total_bytes)))

    print(f"Totale dati stimati: {bytes_to_gb(total_data_bytes):.2f} GB")

    # Recupera indici della tabella con dimensione
    cursor.execute(f"""
        SELECT 
            indexname, 
            pg_relation_size(indexrelid) AS size_bytes
        FROM pg_indexes
        JOIN pg_class ON pg_class.relname = indexname
        JOIN pg_index ON pg_class.oid = pg_index.indexrelid
        WHERE tablename = %s
    """, (table,))
    index_data = cursor.fetchall()

    total_index_bytes = 0
    print("\n📦 Indici:")
    print("{:<30} {:>12}".format("Indice", "Size (GB)"))
    for index_name, size_bytes in index_data:
        total_index_bytes += size_bytes
        print("{:<30} {:>12.2f}".format(index_name, bytes_to_gb(size_bytes)))

    print(f"Totale indici: {bytes_to_gb(total_index_bytes):.2f} GB")
    print(f"Totale complessivo stimato: {bytes_to_gb(total_data_bytes + total_index_bytes):.2f} GB\n")
    print("─" * 60)

cursor.close()
bigbrotr.close()

📊 Analisi dello spazio per tabella:

🧾 TABELLA: events
Numero righe: 208570390
Colonna               Avg (bytes)  Totale (GB)
id                          65.00        12.63
pubkey                      65.00        12.63
created_at                   8.00         1.55
kind                         4.00         0.78
tags                       290.18        56.37
content                    357.26        69.40
sig                        132.00        25.64
Totale dati stimati: 178.99 GB

📦 Indici:
Indice                            Size (GB)
events_pkey                           24.26
idx_events_pubkey                      4.39
idx_events_created_at                  3.56
idx_events_kind                        1.71
idx_events_kind_created_at             6.60
Totale indici: 40.53 GB
Totale complessivo stimato: 219.51 GB

────────────────────────────────────────────────────────────
🧾 TABELLA: relays
Numero righe: 8865
Colonna               Avg (bytes)  Totale (GB)
url                         30.